In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import xml.etree.ElementTree as ET
from lxml import objectify
import pandas as pd
import sys
import scipy
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords
import string
import sklearn.cross_validation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from  sklearn.cluster import KMeans
from sklearn.multiclass  import OneVsRestClassifier
from sklearn import svm
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_selection import SelectKBest, chi2
from stemming.porter2 import stem

In [4]:
cachedStopWords = stopwords.words("english")
toker = RegexpTokenizer(r'(\W)+', gaps=True)
stemmer = SnowballStemmer("english")

def findTk(s):
    s = re.sub("[-a-zA-Z0-9._]+@[-a-zA-Z0-9_]+.[a-zA-Z0-9_.]+", "EmailAddress", s)
    s = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', "WebAddress", s)
    s = re.sub("((?!-)[A-Za-z0-9-]{1,63}(?<!-)\.)+[A-Za-z]{2,6}", "InetDomain", s)
    s = re.sub("{?[A-Fa-f0-9]{8}(?:-[A-Fa-f0-9]{4}){3}-[A-Fa-f0-9]{12}}?", "CorrelIDGuid", s)
    s = re.sub("\d?\d/\d?\d/\d\d\d\d \d?\d \d?\d(\s\d?\d)?(\s(AM|PM))?", "DateTime", s)
    s = re.sub("15\.\d\d?\.\d\d?\d?\d?(\.\d?\d?\d?\d?)?", "ProdVersion", s)
    return s

def cleanTxt(s, tag):
    s = re.sub('(?<=\s)[-+]?[0-9]*\.?[0-9]+(?=\s)', "NumericData", s)
    tokens = toker.tokenize(s.lower())
    r = ""
    for t in tokens:
        i = 0
        if not t in cachedStopWords:
            if re.match("[0-9]+", t)==None:
                st = stemmer.stem(t)
                if len(tag)>0:
                    r = r + " " + tag + st
                r = r + " " + st
    return r.strip()

In [19]:
df_tr = pd.read_csv('Office365 Training Data.tsv', sep='\t', encoding='utf-8')

df_tr['Route_Label'] = df_tr[['CST_1','CST_2']].apply(lambda x:str(x[0])+'||'+str(x[1]), axis=1)
l = df_tr['Route_Label'].unique()  
d_CST2cst = {k:v for (k, v) in zip(l, range(len(l)))}
f = open("O365Support_cst2.pk", 'wb')
pickle.dump(d_CST2cst,f)              
f.close()
d_cst2CST = {v:k for (k, v) in zip(l, range(len(l)))}
f = open("O365Support_2cst.pk", 'wb')
pickle.dump(d_cst2CST,f)              
f.close()
df_tr['Route_Label']=df_tr['Route_Label'].apply(lambda x:d_CST2cst[x])

          
df_tr.fillna('',inplace=True)
df_tr['title'] = df_tr.title.apply(lambda x:findTk(x))
df_tr['title'] = df_tr.title.apply(lambda x:cleanTxt(x,"t_"))
df_tr['problem'] = df_tr.problem.apply(lambda x:findTk(x))
df_tr['problem'] = df_tr.problem.apply(lambda x:cleanTxt(x,"p_"))
df_tr['error'] = df_tr.error.apply(lambda x:findTk(x))
df_tr['error'] = df_tr.error.apply(lambda x:cleanTxt(x,"e_"))
df_tr['txt'] = df_tr[['title','problem','error']].apply(lambda x:str(x[0])+' '+str(x[1])+' '+str(x[2]), axis=1)

df_tr['ist1']= df_tr[['IST_1']].apply(lambda x:x[0], axis=1)
l = df_tr['ist1'].unique()  
d_IST2ist = {k:v for (k, v) in zip(l, range(1,1+len(l)))}
df_tr['ist1']=df_tr['ist1'].apply(lambda x:d_IST2ist[x])   

f = open("O365Support_ist1.pk", 'wb')
pickle.dump(d_IST2ist,f)              
f.close()

df_tr['ist2']= df_tr[['IST_2']].apply(lambda x:x[0], axis=1)
l = df_tr['ist2'].unique()  
d_IST2ist = {k:v for (k, v) in zip(l, range(1,1+len(l)))}
df_tr['ist2']=df_tr['ist2'].apply(lambda x:d_IST2ist[x]) 

f = open("O365Support_ist2.pk", 'wb')
pickle.dump(d_IST2ist,f)              
f.close()

f = open("O365Support_Encoder.pk", 'wb')
enc = OneHotEncoder()
Z = enc.fit_transform(df_tr[['ist1','ist2']].values)
pickle.dump(enc,f)              
f.close()

f = open("O365Support_cv1.pk", 'wb')
cv1 = TfidfVectorizer(ngram_range=(1,2), min_df=0.0001)
W = cv1.fit_transform(df_tr.txt)
pickle.dump(cv1,f)              
f.close()

X_tr = scipy.sparse.hstack((W,Z), format='csr')
Y_tr = df_tr.Route_Label.values

ch2 = SelectKBest(chi2, k=15000)
X_tr = ch2.fit_transform(X_tr, Y_tr)
f = open("O365Support_Chi2.pk", 'wb')
pickle.dump(ch2,f)              
f.close()

clf1 = LinearSVC(C=0.25, tol=1e-10)
clf1.fit(X_tr, Y_tr)
f = open("O365Support_clf1.pk", 'wb')
pickle.dump(clf1,f)              
f.close()

In [20]:
df_tt = pd.read_csv('Office365 Testing Data without Labels.tsv', sep='\t', encoding='utf-8')

df_tt.fillna('',inplace=True)
df_tt['title'] = df_tt.title.apply(lambda x:findTk(x))
df_tt['title'] = df_tt.title.apply(lambda x:cleanTxt(x,"t_"))
df_tt['problem'] = df_tt.problem.apply(lambda x:findTk(x))
df_tt['problem'] = df_tt.problem.apply(lambda x:cleanTxt(x,"p_"))
df_tt['error'] = df_tt.error.apply(lambda x:findTk(x))
df_tt['error'] = df_tt.error.apply(lambda x:cleanTxt(x,"e_"))
df_tt['txt'] = df_tt[['title','problem','error']].apply(lambda x:str(x[0])+' '+str(x[1])+' '+str(x[2]), axis=1)

f = open("O365Support_ist1.pk", 'rb')
d_IST2ist = pickle.load(f)
f.close()
df_tt['ist1']= df_tt[['IST_1']].apply(lambda x:x[0], axis=1)
df_tt['ist1']=df_tt['ist1'].apply(lambda x:d_IST2ist[x] if x in d_IST2ist.keys() else 0)   

f = open("O365Support_ist2.pk", 'rb')
d_IST2ist = pickle.load(f)
f.close()
df_tt['ist2']= df_tt[['IST_2']].apply(lambda x:x[0], axis=1)
df_tt['ist2']=df_tt['ist2'].apply(lambda x:d_IST2ist[x] if x in d_IST2ist.keys() else 0)   


f = open("O365Support_Encoder.pk", 'rb')
enc = pickle.load(f)
Z = enc.transform(df_tt[['ist1','ist2']].values)
f.close()

f = open("O365Support_cv1.pk", 'rb')
cv1 = pickle.load(f)
W = cv1.transform(df_tt.txt)
f.close()

X_tt = scipy.sparse.hstack((W,Z), format='csr')
f = open("O365Support_Chi2.pk", 'rb')
ch2 = pickle.load(f)
f.close()
X_tt = ch2.transform(X_tt)

f = open("O365Support_clf1.pk", 'rb')
clf1 = pickle.load(f)
f.close()

In [21]:
f = open("O365Support_2cst.pk", 'rb')
d_cst2CST = pickle.load(f)
f.close()
df_tt['Scored Labels'] = [d_cst2CST[x] for x in clf1.predict(X_tt)]
df_tt['CST_1'] = df_tt['Scored Labels'].apply(lambda x:x.split('||')[0])
df_tt['CST_2'] = df_tt['Scored Labels'].apply(lambda x:x.split('||')[1])

In [22]:
df_tt.to_csv('output.csv', index=False, sep="\t", columns=['SRId','CST_1','CST_2'])